In [3]:
import optimistix as optx
import jax.numpy as jnp

In [7]:
def G(q, args):
    return (q**3)/3 + (2/9)*q**(9/2)

solver = optx.Newton(rtol=1e-5, atol=1e-5)
sol = optx.fixed_point(G, solver, 0.1)


E0207 16:38:22.461132 3859002 pjrt_stream_executor_client.cc:3085] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Failed to load in-memory CUBIN (compiled for a different GPU?).: CUDA_ERROR_OUT_OF_MEMORY: out of memory


XlaRuntimeError: RESOURCE_EXHAUSTED: Failed to load in-memory CUBIN (compiled for a different GPU?).: CUDA_ERROR_OUT_OF_MEMORY: out of memory